In [ ]:
"""
Created on Wed Aug 20 17:30 2022

Script to tune simple parameterisations in ALL (best-estimate), CV (cross-validation) and BT (bootstrap)

@author: Clara Burgard

"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import multimelt.useful_functions as uf
import multimelt.melt_functions as meltf
from multimelt.constants import *
from scipy import stats
from scipy import linalg

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path='/bettik/burgardc/'
outputpath_simple_all = home_path+'DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_06161821_oneFRIS/'

In [ ]:
bottom = False

In [ ]:
nisf_list_orig = [10, 11, 12, 13, 18, 22, 23, 24, 25, 30, 31, 33, 38, 39, 40, 42, 43, 44, 45, 47, 48, 
                  51, 52, 53, 54, 55, 58, 61, 65, 66, 69, 70, 71, 73, 75]
run_list = ['OPM006','OPM016','OPM018','OPM021']


ALL (tune over the whole sample) => best-estimate parameters

In [ ]:
# PREPARE DATA

file_isf_list = []
thermal_forcing_term_list = []
target_melt_list = []
isf_area_lit = []
GL_flux_list = []
weights_list = []

for n,nemo_run in enumerate(run_list):

    # File mask
    inputpath_mask = home_path+'DATA/BASAL_MELT_PARAM/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    file_isf_list.append(file_isf)

    # File for param
    outputpath_simple = home_path+'DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
    if bottom:
        thermal_forcing_2D = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_bottom_for_tuning_corrected_oneFRIS.nc')
        thermal_forcing_file = xr.open_dataset(outputpath_simple+'thermal_forcing_term_bottom_for_linreg_corrected_oneFRIS.nc')
    else:
        thermal_forcing_2D = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_for_tuning_corrected_oneFRIS.nc')
        thermal_forcing_file = xr.open_dataset(outputpath_simple+'thermal_forcing_term_for_linreg_corrected_oneFRIS.nc')

    thermal_forcing_term = thermal_forcing_file['thermal_forcing_term'].sel(Nisf=file_isf.Nisf)
    thermal_forcing_term = thermal_forcing_term.assign_coords({'time': np.arange(1,len(thermal_forcing_term.time)+1)+n*50})
    thermal_forcing_term_list.append(thermal_forcing_term)

    # File for target
    outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
    NEMO_melt_rates_1D = xr.open_dataset(outputpath_melt+'melt_rates_1D_NEMO_oneFRIS.nc')
    target_melt_Gt_yr = NEMO_melt_rates_1D['melt_Gt_per_y_tot'].sel(Nisf=file_isf.Nisf).sel(time=thermal_forcing_file.time)
    target_melt_Gt_yr = target_melt_Gt_yr.assign_coords({'time': np.arange(1,len(target_melt_Gt_yr.time)+1)+n*50})
    target_melt_list.append(target_melt_Gt_yr)
    
    # Mask of ice shelves
    isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask'] == file_isf.Nisf).sum('Nisf')
    isf_mask = isf_mask.where(isf_mask)

file_isf_all = xr.concat(file_isf_list, dim='nemo_run')
#file_isf_all = file_isf_all.assign_coords({'nemo_run': run_list})

thermal_forcing_all = xr.concat(thermal_forcing_term_list, dim='time')
#thermal_forcing_all = thermal_forcing_all.assign_coords({'nemo_run': run_list})

target_melt_all = xr.concat(target_melt_list, dim='time')
#target_melt_all = target_melt_all.assign_coords({'nemo_run': run_list})

# CONSTANTS AND TARGETS

xx = file_isf_all.x
yy = file_isf_all.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area = abs(dx*dy)

# We need the big constant (which will be multiplied by thermal_forcing_factor and gamma to reach the integrated melt)
big_constant_linear = rho_i * 10**-12 * yearinsec * grid_cell_area * melt_factor
big_constant_quad = (c_po / L_i) * beta_coeff_lazero * (g/(2*f_coriolis))

# the thermal forcing term just missing the gamma (big_constant_quad is added later for the quadratic params)
thermal_forcing_plus = thermal_forcing_all * big_constant_linear 

target_melt_Gt_yr = target_melt_all 

nisf_list = nisf_list_orig

# LINEAR REGRESSION
# prepare data
thermal_forcing_term_stacked = thermal_forcing_plus.sel(Nisf=nisf_list).stack(reg_coord=('Nisf', 'time'))  
target_melt_stacked = target_melt_Gt_yr.sel(Nisf=nisf_list).stack(reg_coord=('Nisf', 'time'))  

# choose domain and param
metrics_all = None

for dom in thermal_forcing_term_stacked.profile_domain:
    metrics_list = [ ]

    for mparam in thermal_forcing_term_stacked.param.values:

        if bottom:
            mparam0 = mparam[:-7:]
        else:
            mparam0 = mparam

        if mparam0 != 'linear_local': # multiply by yet another constant
            thermal_forcing_to_reg_stacked = thermal_forcing_term_stacked.sel(profile_domain=dom,param=mparam) * big_constant_quad
        else:
            thermal_forcing_to_reg_stacked = thermal_forcing_term_stacked.sel(profile_domain=dom,param=mparam)

        if mparam0 != 'linear_local': # multiply by yet another constant
            thermal_forcing_to_plot = thermal_forcing_plus.sel(Nisf=nisf_list).sel(profile_domain=dom,param=mparam) * big_constant_quad
        else:
            thermal_forcing_to_plot = thermal_forcing_plus.sel(Nisf=nisf_list).sel(profile_domain=dom,param=mparam)

        # conduct the regression and compute the metrics
        # we choose lstsq to force the intercept to be 0
        res, resid, rnk, s = linalg.lstsq(thermal_forcing_to_reg_stacked.values[:, np.newaxis], 
                                          target_melt_stacked.values)



        metrics_da = xr.DataArray(data=res.squeeze()).to_dataset(name='slope')

        metrics_list.append(metrics_da)

    metrics_dom = xr.concat(metrics_list, dim='param')
    metrics_dom = metrics_dom.assign_coords({'param': thermal_forcing_term_stacked.param})

    metrics_all = meltf.merge_over_dim(metrics_dom, metrics_all, 'profile_domain', dom.values)

metrics_all.to_netcdf(outputpath_simple_all + 'gammas_simple_ALL.nc')

In [ ]:
# print results for check
dom = 1000
for mparam in metrics_all.param:
    print(str(mparam.values)+' '+str(metrics_all['slope'].sel(param=mparam).sel(profile_domain=dom).values))

CROSS-VALIDATION OVER TIME BLOCKS

In [ ]:
metrics_tt = None
for tt in tqdm(range(1,14)):
    
    # DEFINE TIME BLOCK
    
    tblock_out = tt
    if tblock_out == 0:
        tblock_out, tblock_run, tblock_start, tblock_end = [False, False, False, False]

    if tblock_out:
        inputpath_chunk_info = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/T_S_PROF/'
        info_file = pd.read_csv(inputpath_chunk_info+'info_chunks.txt',header=None, index_col=0)
        tblock_run, tblock_start, tblock_end = info_file.loc[tblock_out]

    # PREPARE DATA

    file_isf_list = []
    thermal_forcing_term_list = []
    target_melt_list = []
    isf_area_lit = []
    GL_flux_list = []
    weights_list = []

    for n,nemo_run in enumerate(run_list):

        # File mask
        inputpath_mask = home_path+'DATA/BASAL_MELT_PARAM/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
        file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
        nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
        file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
        large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
        file_isf = file_isf_nonnan.sel(Nisf=large_isf)
        file_isf_list.append(file_isf)

        # File for param
        outputpath_simple = home_path+'DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
        if bottom:
            thermal_forcing_2D = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_bottom_for_tuning_corrected_oneFRIS.nc')
            thermal_forcing_file = xr.open_dataset(outputpath_simple+'thermal_forcing_term_bottom_for_linreg_corrected_oneFRIS.nc')
        else:
            thermal_forcing_2D = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_for_tuning_corrected_oneFRIS.nc')
            thermal_forcing_file = xr.open_dataset(outputpath_simple+'thermal_forcing_term_for_linreg_corrected_oneFRIS.nc')
        if nemo_run == tblock_run:
            thermal_forcing_term = thermal_forcing_file['thermal_forcing_term'].sel(Nisf=file_isf.Nisf).drop_sel(time=range(tblock_start,tblock_end+1))
        else:
            thermal_forcing_term = thermal_forcing_file['thermal_forcing_term'].sel(Nisf=file_isf.Nisf)
        thermal_forcing_term = thermal_forcing_term.assign_coords({'time': np.arange(1,len(thermal_forcing_term.time)+1)+n*50})
        thermal_forcing_term_list.append(thermal_forcing_term)

        # File for target
        outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
        NEMO_melt_rates_1D = xr.open_dataset(outputpath_melt+'melt_rates_1D_NEMO_oneFRIS.nc')
        if nemo_run == tblock_run:
            target_melt_Gt_yr = NEMO_melt_rates_1D['melt_Gt_per_y_tot'].sel(Nisf=file_isf.Nisf).sel(time=thermal_forcing_file.time).drop_sel(time=range(tblock_start,tblock_end+1))
        else:
            target_melt_Gt_yr = NEMO_melt_rates_1D['melt_Gt_per_y_tot'].sel(Nisf=file_isf.Nisf).sel(time=thermal_forcing_file.time)
        target_melt_Gt_yr = target_melt_Gt_yr.assign_coords({'time': np.arange(1,len(target_melt_Gt_yr.time)+1)+n*50})
        target_melt_list.append(target_melt_Gt_yr)

        isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask'] == file_isf.Nisf).sum('Nisf')
        isf_mask = isf_mask.where(isf_mask)

    file_isf_all = xr.concat(file_isf_list, dim='nemo_run')
    #file_isf_all = file_isf_all.assign_coords({'nemo_run': run_list})

    thermal_forcing_all = xr.concat(thermal_forcing_term_list, dim='time')
    #thermal_forcing_all = thermal_forcing_all.assign_coords({'nemo_run': run_list})

    target_melt_all = xr.concat(target_melt_list, dim='time')
    #target_melt_all = target_melt_all.assign_coords({'nemo_run': run_list})
    
    # CONSTANTS AND TARGETS
    
    xx = file_isf_all.x
    yy = file_isf_all.y
    dx = (xx[2] - xx[1]).values
    dy = (yy[2] - yy[1]).values
    grid_cell_area = abs(dx*dy)
    
    # We need the big constant
    big_constant_linear = rho_i * 10**-12 * yearinsec * grid_cell_area * melt_factor
    big_constant_quad = (c_po / L_i) * beta_coeff_lazero * (g/(2*f_coriolis))
    
    # the thermal forcing term just missing the gamma (big_constant_quad is added later)
    thermal_forcing_plus = thermal_forcing_all * big_constant_linear 
    
    target_melt_Gt_yr = target_melt_all 
    
    nisf_list = nisf_list_orig
    
    # LINEAR REGRESSION
    # prepare data
    thermal_forcing_term_stacked = thermal_forcing_plus.sel(Nisf=nisf_list).stack(reg_coord=('Nisf', 'time'))  
    target_melt_stacked = target_melt_Gt_yr.sel(Nisf=nisf_list).stack(reg_coord=('Nisf', 'time'))  
    
    # choose domain and param
    metrics_all = None

    for dom in thermal_forcing_term_stacked.profile_domain:
        metrics_list = [ ]

        for mparam in thermal_forcing_term_stacked.param.values:

            if bottom:
                mparam0 = mparam[:-7:]
            else:
                mparam0 = mparam

            if mparam0 != 'linear_local':
                thermal_forcing_to_reg_stacked = thermal_forcing_term_stacked.sel(profile_domain=dom,param=mparam) * big_constant_quad
            else:
                thermal_forcing_to_reg_stacked = thermal_forcing_term_stacked.sel(profile_domain=dom,param=mparam)

            if mparam0 != 'linear_local':
                thermal_forcing_to_plot = thermal_forcing_plus.sel(Nisf=nisf_list).sel(profile_domain=dom,param=mparam) * big_constant_quad
            else:
                thermal_forcing_to_plot = thermal_forcing_plus.sel(Nisf=nisf_list).sel(profile_domain=dom,param=mparam)

            # conduct the regression and compute the metrics
            res, resid, rnk, s = linalg.lstsq(thermal_forcing_to_reg_stacked.values[:, np.newaxis], 
                                              target_melt_stacked.values)



            metrics_da = xr.DataArray(data=res.squeeze()).to_dataset(name='slope')

            metrics_list.append(metrics_da)

        metrics_dom = xr.concat(metrics_list, dim='param')
        metrics_dom = metrics_dom.assign_coords({'param': thermal_forcing_term_stacked.param})

        metrics_all = meltf.merge_over_dim(metrics_dom, metrics_all, 'profile_domain', dom.values)

    metrics_tt = meltf.merge_over_dim(metrics_all, metrics_tt, 'tblock', tt)

metrics_tt.to_netcdf(outputpath_simple_all + 'gammas_simple_CV_time.nc')

CROSS-VALIDATION ACROSS ICE SHELVES

In [ ]:
metrics_isf = None
for isf_out in tqdm(nisf_list_orig):    

    # PREPARE DATA

    file_isf_list = []
    thermal_forcing_term_list = []
    target_melt_list = []
    isf_area_lit = []
    GL_flux_list = []
    weights_list = []

    for n,nemo_run in enumerate(run_list):

        # File mask
        inputpath_mask = home_path+'DATA/BASAL_MELT_PARAM/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
        file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
        nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
        file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
        large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
        isf_of_int = large_isf.drop_sel(Nisf=isf_out)
        file_isf = file_isf_nonnan.sel(Nisf=isf_of_int)
        file_isf_list.append(file_isf)

        # File for param
        outputpath_simple = home_path+'DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
        if bottom:
            thermal_forcing_2D = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_bottom_for_tuning_corrected_oneFRIS.nc')
            thermal_forcing_file = xr.open_dataset(outputpath_simple+'thermal_forcing_term_bottom_for_linreg_corrected_oneFRIS.nc')
        else:
            thermal_forcing_2D = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_for_tuning_corrected_oneFRIS.nc')
            thermal_forcing_file = xr.open_dataset(outputpath_simple+'thermal_forcing_term_for_linreg_corrected_oneFRIS.nc')
        thermal_forcing_term = thermal_forcing_file['thermal_forcing_term'].sel(Nisf=isf_of_int)
        thermal_forcing_term = thermal_forcing_term.assign_coords({'time': np.arange(1,len(thermal_forcing_term.time)+1)+n*50})
        thermal_forcing_term_list.append(thermal_forcing_term)

        # File for target
        outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
        NEMO_melt_rates_1D = xr.open_dataset(outputpath_melt+'melt_rates_1D_NEMO_oneFRIS.nc')
        target_melt_Gt_yr = NEMO_melt_rates_1D['melt_Gt_per_y_tot'].sel(Nisf=isf_of_int).sel(time=thermal_forcing_file.time)
        target_melt_Gt_yr = target_melt_Gt_yr.assign_coords({'time': np.arange(1,len(target_melt_Gt_yr.time)+1)+n*50})
        target_melt_list.append(target_melt_Gt_yr)

        isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask'] == isf_of_int).sum('Nisf')
        isf_mask = isf_mask.where(isf_mask)

    file_isf_all = xr.concat(file_isf_list, dim='nemo_run')
    #file_isf_all = file_isf_all.assign_coords({'nemo_run': run_list})

    thermal_forcing_all = xr.concat(thermal_forcing_term_list, dim='time')
    #thermal_forcing_all = thermal_forcing_all.assign_coords({'nemo_run': run_list})

    target_melt_all = xr.concat(target_melt_list, dim='time')
    #target_melt_all = target_melt_all.assign_coords({'nemo_run': run_list})
    
    # CONSTANTS AND TARGETS
    
    xx = file_isf_all.x
    yy = file_isf_all.y
    dx = (xx[2] - xx[1]).values
    dy = (yy[2] - yy[1]).values
    grid_cell_area = abs(dx*dy)
    
    # We need the big constant
    big_constant_linear = rho_i * 10**-12 * yearinsec * grid_cell_area * melt_factor
    big_constant_quad = (c_po / L_i) * beta_coeff_lazero * (g/(2*f_coriolis))
    
    # the thermal forcing term just missing the gamma (big_constant_quad is added later)
    thermal_forcing_plus = thermal_forcing_all * big_constant_linear 
    
    target_melt_Gt_yr = target_melt_all 
    
    nisf_list = isf_of_int
    
    # LINEAR REGRESSION
    # prepare data
    thermal_forcing_term_stacked = thermal_forcing_plus.sel(Nisf=nisf_list).stack(reg_coord=('Nisf', 'time'))  
    target_melt_stacked = target_melt_Gt_yr.sel(Nisf=nisf_list).stack(reg_coord=('Nisf', 'time'))  
    
    # choose domain and param
    metrics_all = None

    for dom in thermal_forcing_term_stacked.profile_domain:
        metrics_list = [ ]

        for mparam in thermal_forcing_term_stacked.param.values:

            if bottom:
                mparam0 = mparam[:-7:]
            else:
                mparam0 = mparam

            if mparam0 != 'linear_local':
                thermal_forcing_to_reg_stacked = thermal_forcing_term_stacked.sel(profile_domain=dom,param=mparam) * big_constant_quad
            else:
                thermal_forcing_to_reg_stacked = thermal_forcing_term_stacked.sel(profile_domain=dom,param=mparam)

            if mparam0 != 'linear_local':
                thermal_forcing_to_plot = thermal_forcing_plus.sel(Nisf=nisf_list).sel(profile_domain=dom,param=mparam) * big_constant_quad
            else:
                thermal_forcing_to_plot = thermal_forcing_plus.sel(Nisf=nisf_list).sel(profile_domain=dom,param=mparam)

            # conduct the regression and compute the metrics
            res, resid, rnk, s = linalg.lstsq(thermal_forcing_to_reg_stacked.values[:, np.newaxis], 
                                              target_melt_stacked.values)



            metrics_da = xr.DataArray(data=res.squeeze()).to_dataset(name='slope')

            metrics_list.append(metrics_da)

        metrics_dom = xr.concat(metrics_list, dim='param')
        metrics_dom = metrics_dom.assign_coords({'param': thermal_forcing_term_stacked.param})

        metrics_all = meltf.merge_over_dim(metrics_dom, metrics_all, 'profile_domain', dom.values)

    metrics_isf = meltf.merge_over_dim(metrics_all, metrics_isf, 'isf', isf_out)

metrics_isf.to_netcdf(outputpath_simple_all + 'gammas_simple_CV_shelves.nc')

BOOTSTRAPPING

In [ ]:
inputpath_chunk_info = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/T_S_PROF/'
info_file = pd.read_csv(inputpath_chunk_info+'info_chunks.txt',header=None, index_col=0)

# PREPARE DATA

file_isf_list = []
thermal_forcing_term_list = []
target_melt_list = []
time_list = []

for n,nemo_run in enumerate(run_list):

    # File mask
    inputpath_mask = home_path+'DATA/BASAL_MELT_PARAM/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    file_isf_list.append(file_isf)

    # File for param
    outputpath_simple = home_path+'DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
    if bottom:
        thermal_forcing_2D = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_bottom_for_tuning_corrected_oneFRIS.nc')
        thermal_forcing_file = xr.open_dataset(outputpath_simple+'thermal_forcing_term_bottom_for_linreg_corrected_oneFRIS.nc')
    else:
        thermal_forcing_2D = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_for_tuning_corrected_oneFRIS.nc')
        thermal_forcing_file = xr.open_dataset(outputpath_simple+'thermal_forcing_term_for_linreg_corrected_oneFRIS.nc')

    thermal_forcing_term_orig = thermal_forcing_file['thermal_forcing_term'].sel(Nisf=file_isf.Nisf)
    thermal_forcing_term = thermal_forcing_term_orig.assign_coords({'time': np.arange(1,len(thermal_forcing_term_orig.time)+1)+n*50})
    thermal_forcing_term_list.append(thermal_forcing_term)
    time_list.append(thermal_forcing_term_orig.time.values)


    # File for target
    outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
    NEMO_melt_rates_1D = xr.open_dataset(outputpath_melt+'melt_rates_1D_NEMO_oneFRIS.nc')
    target_melt_Gt_yr = NEMO_melt_rates_1D['melt_Gt_per_y_tot'].sel(Nisf=file_isf.Nisf).sel(time=thermal_forcing_file.time)
    target_melt_Gt_yr = target_melt_Gt_yr.assign_coords({'time': np.arange(1,len(target_melt_Gt_yr.time)+1)+n*50})
    target_melt_list.append(target_melt_Gt_yr)

    isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask'] == file_isf.Nisf).sum('Nisf')
    isf_mask = isf_mask.where(isf_mask)

file_isf_all = xr.concat(file_isf_list, dim='nemo_run')
#file_isf_all = file_isf_all.assign_coords({'nemo_run': run_list})

thermal_forcing_all = xr.concat(thermal_forcing_term_list, dim='time')
#thermal_forcing_all = thermal_forcing_all.assign_coords({'nemo_run': run_list})

target_melt_all = xr.concat(target_melt_list, dim='time')
#target_melt_all = target_melt_all.assign_coords({'nemo_run': run_list})

yy_list = np.concatenate( time_list, axis=0 )

n = 0
n_list = ['OPM006']
for tt in range(1,len(thermal_forcing_all.time)):
    if (thermal_forcing_all.time[tt] - thermal_forcing_all.time[tt-1]) > 1:
        n = n + 1
    n_list.append(run_list[n])

res_da_time = xr.DataArray(data=yy_list, dims= 'time', coords={'time': thermal_forcing_all.time}).to_dataset(name='years')
res_da_runs = xr.DataArray(data=n_list, dims= 'time', coords={'time': thermal_forcing_all.time}).to_dataset(name='run')
idx_ds = xr.merge([res_da_time,res_da_runs])

# CONSTANTS AND TARGETS

xx = file_isf_all.x
yy = file_isf_all.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area = abs(dx*dy)

# We need the big constant
big_constant_linear = rho_i * 10**-12 * yearinsec * grid_cell_area * melt_factor
big_constant_quad = (c_po / L_i) * beta_coeff_lazero * (g/(2*f_coriolis))

# the thermal forcing term just missing the gamma (big_constant_quad is added later)
thermal_forcing_plus = thermal_forcing_all * big_constant_linear 

target_melt_Gt_yr = target_melt_all 

tblock_dim = range(1,14)
isf_dim = [10,11,12,13,18,22,23,24,25,30,31,33,38,39,40,42,43,44,45,47,48,51,52,53,54,55,58,61,65,66,69,70,71,73,75]

In [ ]:
iteration = 15 # every iteration runs through 1000 samples (I went through iteration 1 to 15)

# LOOP OVER BOOTSTRAP
metrics_uncertainty = None
all_isf_samples = None
all_time_samples = None

for n in tqdm(range(1000)):    
    
    np.random.seed((iteration-1)*1000+n)

    # define the random bootstrap samples
    random_tblock_sample = np.sort(np.random.choice(tblock_dim, size=len(tblock_dim), replace=True))
    info_t = info_file.loc[random_tblock_sample].to_numpy()

    random_time_list = []
    rrun_list = []
    for tt in random_tblock_sample:
        run, tstart, tend = info_file.loc[tt]
        random_time_list.append(idx_ds['time'].where((idx_ds['run']==run) & (idx_ds['years']>=tstart) & (idx_ds['years']<=tend)).dropna(dim='time').values.astype(int))
        rrun_list.append(run)
    time_idx = np.concatenate(random_time_list,axis=0)

    final_run_list = []
    for rr in run_list:
        if rr in rrun_list:
            final_run_list.append(rr)

    random_isf_sample = np.random.choice(isf_dim, size=len(isf_dim), replace=True)
        
    # prepare data
    thermal_forcing_term_stacked = thermal_forcing_plus.sel(Nisf=random_isf_sample, time=time_idx).unstack().stack(reg_coord=('Nisf', 'time')).sel(profile_domain=[50,1000])
    target_melt_stacked = target_melt_Gt_yr.sel(Nisf=random_isf_sample, time=time_idx).unstack().stack(reg_coord=('Nisf', 'time'))
    
    # choose domain and param

    metrics_all_unc = None

    for dom in thermal_forcing_term_stacked.profile_domain:

        metrics_list = [ ]

        for mparam in thermal_forcing_term_stacked.param:

            if mparam != 'linear_local':
                thermal_forcing_to_reg_stacked = thermal_forcing_term_stacked.sel(profile_domain=dom,param=mparam)*big_constant_quad
            else:
                thermal_forcing_to_reg_stacked = thermal_forcing_term_stacked.sel(profile_domain=dom,param=mparam)

            # conduct the regression and compute the metrics
            res, resid, rnk, s = linalg.lstsq(thermal_forcing_to_reg_stacked.values[:, np.newaxis], target_melt_stacked.values)

            yhat = thermal_forcing_to_reg_stacked*res
            y = target_melt_stacked

            metrics_da = xr.DataArray(data=res.squeeze()).to_dataset(name='slope')

            metrics_list.append(metrics_da)

        metrics_dom = xr.concat(metrics_list, dim='param')

        metrics_all_unc = meltf.merge_over_dim(metrics_dom, metrics_all_unc, 'profile_domain', dom.values)

    metrics_uncertainty = meltf.merge_over_dim(metrics_all_unc, metrics_uncertainty, 'bootstrap', n)
    
    
    metrics_isf_sample = xr.DataArray(data=random_isf_sample, dims=['isf_dim'], coords={'bootstrap': n}).to_dataset(name='random_isf')
    if len(time_idx) < 130:
        new_time = np.zeros(130) * np.nan
        for tt in range(len(time_idx)):
            new_time[tt] = time_idx[tt]
    metrics_time_sample = xr.DataArray(data=new_time, dims=['time_dim'], coords={'bootstrap': n}).to_dataset(name='random_time')
    
    all_isf_samples = meltf.merge_over_dim(metrics_isf_sample, all_isf_samples, 'bootstrap', n)
    all_time_samples = meltf.merge_over_dim(metrics_time_sample, all_time_samples, 'bootstrap', n)

all_info_metrics = xr.merge([metrics_uncertainty, all_isf_samples, all_time_samples])
all_info_metrics.to_netcdf(outputpath_simple_all+'clusterbootstrap1000_'+str(iteration).zfill(2)+'_timeANDisf.nc')
